# G123 window size calibration

In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.yaml"
cohort_id = "BF-09_Houet_colu_2012_Q3"

## Setup

In [ ]:
import pandas as pd
import numpy as np
import yaml
from selection_atlas.setup import AtlasSetup

# Initialise the atlas setup.
setup = AtlasSetup(config_file)

In [ ]:
df_cohorts = pd.read_csv(setup.cohorts_file).set_index("cohort_id")
df_cohorts.head()

In [ ]:
cohort = df_cohorts.loc[cohort_id]
cohort

In [ ]:
# determine the phasing analysis to use
cohort.taxon

In [ ]:
sample_query = cohort.sample_query
sample_query

In [ ]:
if cohort.taxon == "arabiensis":
    sites = site_mask = "arab"
else:
    sites = site_mask = "gamb_colu"
site_mask

## Run calibration

In [ ]:
setup.malariagen_api.plot_g123_calibration(
    contig=setup.g123_calibration_contig,
    site_mask=site_mask,
    sites=sites,
    sample_sets=setup.sample_sets,
    sample_query=setup.sample_query,
    min_cohort_size=setup.min_cohort_size,
    max_cohort_size=setup.max_cohort_size,
    window_sizes=setup.g123_calibration_window_sizes,
);

In [ ]:
calibration_runs = setup.malariagen_api.g123_calibration(
    contig=setup.g123_calibration_contig,
    site_mask=site_mask,
    sites=sites,
    sample_sets=setup.sample_sets,
    sample_query=setup.sample_query,
    min_cohort_size=setup.min_cohort_size,
    max_cohort_size=setup.max_cohort_size,
    window_sizes=setup.g123_calibration_window_sizes,
)
calibration_runs

In [ ]:
selected_window_size = None
for window_size in window_sizes:
    x = calibration_runs[str(window_size)]
    x95 = np.percentile(x, 95)
    if x95 < 0.1:
        selected_window_size = window_size
        break
selected_window_size

## Write outputs

In [ ]:
output = {"g123_window_size": selected_window_size}
g123_calibration_file = setup.g123_calibration_files.as_posix().format(cohort=cohort_id)
with open(g123_calibration_file, mode="w") as output_file:
    yaml.safe_dump(output, output_file)